Реализуйте задачу классификации на основе BERT-like модели и KNN на данных [Russian Intents Dataset](https://www.kaggle.com/datasets/constantinwerner/qa-intents-dataset-university-domain) с Kaggle.

In [ ]:
!pip install -U accelerate
!pip install -U transformers

# Загрузка и подготовка данных

In [ ]:
import pandas as pd
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
import torch
from transformers import BertTokenizer, BertForSequenceClassification, AdamW
from torch.utils.data import Dataset
from sklearn.preprocessing import LabelEncoder
from transformers import TrainingArguments, Trainer
from sklearn.metrics import accuracy_score
import numpy as np

In [ ]:
import warnings
warnings.simplefilter('ignore')

In [ ]:
train= pd.read_csv('dataset_train.tsv',delimiter='\t',encoding="utf-8",names=['text', 'intent'])
test= pd.read_csv('dataset_test.tsv',delimiter='\t',encoding="utf-8",names=['text', 'intent'])

In [ ]:
train.head()

,text,intent
0,мне нужна справка,statement_general
1,оформить справку,statement_general
2,взять справку,statement_general
3,справку как получить,statement_general
4,справку ммф где получаться,statement_general


In [ ]:
X_train, y_train = train['text'], train['intent']

In [ ]:
test.head()

,text,intent
0,как получить справку,statement_general
1,мне нужна справка,statement_general
2,справка студента эф петь,conform
3,справка студента фф оформлять,conform
4,как мне заказать справка об обучении,conform


In [ ]:
X_test, y_test = test['text'], test['intent']

In [ ]:
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [ ]:
lemmatizer = WordNetLemmatizer()
stop_words = set(stopwords.words('russian'))

In [ ]:
def preprocess_text(text):
    tokens = word_tokenize(text)
    tokens = [lemmatizer.lemmatize(token) for token in tokens if token.isalnum()]
    tokens = [token.lower() for token in tokens if token.lower() not in stop_words]
    return ' '.join(tokens)

In [ ]:
X_train = X_train.apply(preprocess_text)

In [ ]:
X_test = X_test.apply(preprocess_text)

In [ ]:
model_name = 'bert-base-multilingual-cased'
tokenizer = BertTokenizer.from_pretrained(model_name)

In [ ]:
le = LabelEncoder()
y_train = le.fit_transform(y_train)
y_test = le.transform(y_test)

In [ ]:
class IntentDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_length):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = self.texts[idx]
        label = self.labels[idx]
        encoding = self.tokenizer(text, return_tensors='pt', max_length=self.max_length, padding='max_length', truncation=True)
        return {'input_ids': encoding['input_ids'].flatten(), 'attention_mask': encoding['attention_mask'].flatten(), 'label': torch.tensor(label)}

In [ ]:
max_length = 128
batch_size = 32

train_dataset = IntentDataset(X_train, y_train, tokenizer, max_length)
val_dataset = IntentDataset(X_test, y_test, tokenizer, max_length)

# Обучение модели

## Bert

Bert, или Bidirectional Encoder Representations from Transformers, является одной из самых популярных моделей для обработки естественного языка (NLP). Она была представлена в 2018 году командой Google AI и является прорывом в области представления текстовых данных.

Основная идея модели Bert заключается в использовании предобучения на большом объеме текстовых данных и последующей дообучении ее для выполнения конкретных задач NLP, таких как классификация, вопросно-ответная система, именованное сущностное распознавание и другие.

Одной из ключевых особенностей Bert является его способность «понимать» контекст в тексте с использованием двухстороннего (bidirectional) подхода. Он использует трансформерную архитектуру, которая работает на основе механизмов внимания (attention mechanism) для выявления связей между словами в предложении.

В отличие от предыдущих моделей, которые обрабатывают текст последовательно (слева направо или справа налево), Bert учитывает взаимодействие всех слов в предложении, что позволяет ей получать более богатые представления текста.

Модель Bert имеет несколько вариантов разных размеров, таких как Bert-base, Bert-large и другие. Большие варианты имеют большее количество параметров и повышенную вычислительную сложность, но за счет этого могут показывать более высокую производительность при решении сложных задач NLP.

С помощью предварительно обученной модели Bert вы можете извлекать эмбеддинги (векторные представления) текстов, которые уже содержат семантическую информацию о словах и их контексте. Эти эмбеддинги можно использовать для решения различных задач NLP или дообучения модели на конкретной задаче с помощью дополнительных слоев нейронной сети.

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
model_bert = BertForSequenceClassification.from_pretrained(model_name, num_labels=len(train['intent'].unique()))

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-multilingual-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return {'accuracy' : accuracy_score(predictions, labels)}

In [ ]:
args = TrainingArguments("test-glue",
                         evaluation_strategy = "steps",
                         learning_rate=2e-5,
                         per_device_train_batch_size=32,
                         per_device_eval_batch_size=32,
                         num_train_epochs=10,
                         weight_decay=0.05,
                         load_best_model_at_end=False,
                         report_to=None,
                         metric_for_best_model = 'accuracy')

In [ ]:
trainer = Trainer(
    model_bert,
    args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics)

In [ ]:
 trainer.train()

Step,Training Loss,Validation Loss,Accuracy
500,0.928700,0.455513,0.932050
1000,0.334900,0.214640,0.950170
1500,0.181400,0.147120,0.956965
2000,0.137600,0.146835,0.952435
2500,0.123600,0.144776,0.953567
3000,0.113700,0.142500,0.954700
3500,0.107100,0.133348,0.954700
4000,0.105400,0.135004,0.954700


Step,Training Loss,Validation Loss


TrainOutput(global_step=4140, training_loss=0.24894090320753015, metrics={'train_runtime': 2991.9294, 'train_samples_per_second': 44.219, 'train_steps_per_second': 1.384, 'total_flos': 8713337101056000.0, 'train_loss': 0.24894090320753015, 'epoch': 10.0})

##KNN

Модель K ближайших соседей (K-nearest neighbors, KNN) - это простой алгоритм машинного обучения, используемый для классификации и регрессии. В KNN, объекты представляются в виде точек в пространстве признаков, и классификация нового объекта происходит путем определения его принадлежности к определенному классу на основе ближайших соседей.

Основная идея KNN заключается в том, что если объекты в пространстве признаков близки друг к другу, то они скорее всего принадлежат к одному и тому же классу. В алгоритме KNN происходит поиск K ближайших соседей нового объекта среди тренировочных данных. Критерий близости определяется расстоянием между объектами в пространстве признаков, часто используется евклидово расстояние.

Когда ближайшие соседи найдены, KNN принимает во внимание их метки классов и принимает решение о классификации нового объекта путем голосования большинства. Если речь идет о регрессии, модель может вместо этого вернуть среднее или медианное значение меток ближайших соседей.

Значение параметра K в KNN определяет количество соседей, которые используются для принятия решения. Большое значение K сглаживает решение, тогда как маленькое значение K делает модель более чувствительной к выбросам.

KNN - непараметрический алгоритм, что означает, что он не требует предположений о распределении данных или форме решающей функции. Однако, KNN может быть чувствителен к масштабу признаков и также может потребовать больших вычислительных ресурсов для работы с большими наборами данных.

В целом, KNN - простая, но эффективная модель, которая может быть эффективно использована для классификации и регрессии, особенно в случаях, когда данные имеют небольшие размеры или существует явное пространственное разделение между классами.

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report

In [ ]:
vectorizer =  TfidfVectorizer()

In [ ]:
X_train_tfidf = vectorizer.fit_transform(X_train)

In [ ]:
X_test_tfidf = vectorizer.transform(X_test)

In [ ]:
knn = KNeighborsClassifier(n_neighbors=5)

In [ ]:
knn.fit(X_train_tfidf, y_train)

KNeighborsClassifier()

In [ ]:
y_pred_knn = knn.predict(X_test_tfidf)

In [ ]:
print(classification_report(y_test, y_pred_knn))

              precision    recall  f1-score   support

           0       1.00      0.50      0.67         2
           1       0.86      1.00      0.92         6
           2       0.52      1.00      0.69        12
           3       0.92      1.00      0.96        12
           4       1.00      1.00      1.00         2
           5       1.00      1.00      1.00         3
           6       1.00      1.00      1.00         3
           7       1.00      1.00      1.00         9
           8       1.00      1.00      1.00         4
           9       1.00      1.00      1.00         6
          10       1.00      1.00      1.00         3
          11       1.00      1.00      1.00         3
          12       1.00      1.00      1.00        11
          13       1.00      1.00      1.00         3
          14       1.00      1.00      1.00         2
          15       1.00      1.00      1.00        11
          16       1.00      1.00      1.00         3
          17       1.00    

# Заключение:

Качество классификации текста ("Russian Intents Dataset") методом KNN с использованием меры TF-IDF получилось на достаточно высоком уровне (accuracy = 80%), но хуже, чем с использованием BERT-like модели(accuracy = 95%)